In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
inputs=data.columns[1:9]
target=data.columns[9]
inputs=data[inputs]
target=data[target]
numerical=inputs.select_dtypes(include=np.number).columns.tolist()
category=inputs.select_dtypes(include="object").columns.tolist()

# sns.heatmap(data[numerical].corr(),annot=True)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2,random_state=42)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
inputs[numerical]=scaler.fit_transform(inputs[numerical])
x_test[numerical]=scaler.fit_transform(x_test[numerical])
test[numerical]=scaler.fit_transform(test[numerical])


from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
encoder.fit(data[category])
encoded=list(encoder.get_feature_names_out(category))
inputs[encoded]=encoder.transform(inputs[category])
x_test[encoded]=encoder.transform(x_test[category])
test[encoded]=encoder.transform(test[category])


import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
model=HistGradientBoostingClassifier(max_iter=750000,random_state=10,l2_regularization=15,max_depth=20,learning_rate=0.11234)
from sklearn.metrics import accuracy_score

/tmp/ipython-input-30-2411793156.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs[numerical]=scaler.fit_transform(inputs[numerical])
/tmp/ipython-input-30-2411793156.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs[encoded]=encoder.transform(inputs[category])


In [ ]:
from sklearn.preprocessing import LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(target) # Encode y_train
y_test_encoded = target_encoder.transform(y_test) # Encode y_test
model.fit(inputs[numerical + encoded], y_train_encoded) # Fit model with encoded target
test_pred_proba = model.predict_proba(test[numerical + encoded])
top_3_preds_indices = np.argsort(test_pred_proba, axis=1)[:, ::-1][:, :3]

# Map the indices back to the original fertilizer names
top_3_labels = np.array([target_encoder.classes_[i] for i in top_3_preds_indices.ravel()]).reshape(top_3_preds_indices.shape)

submission = pd.read_csv('sample_submission.csv')
submission = pd.DataFrame({
    'id': submission['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels] # Join strings directly
})

# submission.to_csv('submission.csv', index=False)
# print("✅ Submission file with real fertilizer names saved successfully!")
print(submission.head())

# Calculate accuracy with encoded y_test
accuracy = accuracy_score(y_test_encoded, model.predict(x_test[numerical + encoded]))
print(f"Accuracy: {accuracy}")

       id             Fertilizer Name
0  750000              DAP 20-20 Urea
1  750001     17-17-17 10-26-26 20-20
2  750002        20-20 28-28 10-26-26
3  750003  14-35-14 10-26-26 17-17-17
4  750004         20-20 10-26-26 Urea
Accuracy: 0.27554


In [ ]:
submission.to_csv('submission10.csv', index=False)